In [140]:
from mesa import Agent, Model
import random
import copy

In [323]:
class Man(Agent):
        def __init__(self, unique_id, adol, model):
            super().__init__(unique_id,model)
            self.id = unique_id
            self.adol = adol
            self.age = 0
            self.gender = "M"
            self.married = False
            self.asp = 50
            self.value = random.choice(range(100))

        def step(self):
            self.age += 1
            if self.married:
                return
            partner = self.getMatch()
            if self.age <= self.adol:
                self.updateAsp(partner)
                return
            self.married = self.match
            partner.married = self.match
            self.aom = (self.match and self.age or None)
            partner.aom = (self.match and partner.age or None)
            
        def getMatch(self):
            valid_match = False
            while valid_match == False:
                partner = random.choice(self.model.women)
                if partner.married != True:
                    valid_match = True
                    
            self.makeoffer = bool(self.asp <= partner.value)
            partner.makeoffer = bool(partner.asp <= self.value)
            
            self.getoffer = partner.makeoffer
            partner.getoffer = self.makeoffer
            
            self.match = bool(self.makeoffer == True and self.getoffer == True)
            partner.match = self.match
            
            return partner
            
        def updateAsp(self,partner):
            
            if self.match: #if there is a match
                if partner.value >= self.asp: #and if the woman's value > mans aspiration
                    self.asp = partner.value #replace the mans's aspiration with the man's value
                if self.value >= partner.asp: #and if the mans value > womans aspiration
                    partner.asp = self.value #replace the woman's aspiration with the woman's value
            
            if not self.getoffer: #if the man didnt get an offer
                if self.asp > partner.value: #and his aspiration is higher than the woman's value
                    self.asp = partner.value #change his aspiration to her value
                    
            if not self.makeoffer: #if the man didnt make an offer
                if partner.asp > self.value: #and the woman's aspiration is higher than the mans value
                    partner.asp = self.value #change her aspiration to his value
    
class Woman(Man):
        def __init__(self,unique_id,adol,model):
            super().__init__(unique_id,adol,model)
            self.gender = "F"
        

In [324]:
class MarriageModel(Model):
    
    def __init__(self,N,adol):
        self.num_agents = N
        self.adol = adol
        self.women = []
        self.men = []
        #Create agents
        for i in range(self.num_agents):
            self.women.append(Woman(i,adol,self))
            self.men.append(Man(i,adol,self))
            
            

In [325]:

test = MarriageModel(100, 12)
test

In [326]:
testing = test.women[1]

